# Overall strategy:

#### 1) split into good and bad data. Currently only splitting based on > 50% fordelt. Need to split those who only deliver to 1 bedrift into bad as well. 
#### 2) for the bad data, use ml to guess the oms , distribute drkost based on profit level and oms. 
#### 3) merge this data back with the good data.
#### 4) run the regular AO with 99 on th entire df
#### 5) create an update file. Send to google cloud
#### 6) send to linux. Use the sas code to update isee

### How to organise functions in final notebook

#### Choose naring
#### Choose parameters for separating between good, bad dfs
#### Apply which machine learning model? Varehandels
#### Bring back everything to a final DF. Apply 999 logic to this df. 
#### Further updates to rad 0 level data (foretak) 
#### Need to think about what to do with reg_types for example reg_type 03

- (A) not 100% fordelt. automatically adjust. Create a list of things of foretak to check naring. 
- avanse
- 1000 feils
- IKT investering


- Need to bring in past years data and data from other sources. 

#### Create updatefile. 

## FEATURE ENGINEERING!



In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import pyarrow as pa
import pyarrow.parquet as pq
import gcsfs
import getpass
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import geopandas as gpd

# import sgis as sg
import dapla as dp
import datetime
from dapla.auth import AuthClient
from dapla import FileClient
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import requests
from pyjstat import pyjstat


import sys

sys.path.append("../functions")
import kommune_pop
import kommune_inntekt
import kpi
import ao

fs = FileClient.get_gcs_file_system()
import numpy as np


import warnings

warnings.filterwarnings("ignore")

# good_df = ao.rette_bedrifter(good_df)

# Årgang

In [ ]:
year = 2021
fjor = year - 1
limit = 0.6

# get dynarev data

In [ ]:
fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-noeku-data-produkt/eimerdb/nokubasen/skjemadata/aar={year}/skjema=RA-0174-1/*"
    )
    if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
skjema = table.to_pandas()

In [ ]:
felt_id_values = [
    "V_ORGNR",
    "F_ADRESSE",
    "FJOR_NACE_B_T1",
    "TMP_SN2007_5",
    "B_KOMMUNENR",
    "REGTYPE",
    "B_SYSSELSETTING_SYSS",
    "TMP_NY_BDR_SYSS",
    "GJELDENDE_BDR_SYSS",
    "FJOR_SYSSEL_T1",
    "LONN_PST_AORDN",
    "GJELDENDE_LONN_KR",
    "LONN",
    "FJOR_LONN_KR_T1",
    "TMP_SNITTLONN",
    "FJOR_SNITTLONN_T1",
    "GJELDENDE_OMSETN_KR",
    "OMSETN_KR",
    "FJOR_OMSETN_KR_T1",
    "TMP_SNITTOMS",
    "FJOR_SNITTOMS_T1",
    "TMP_SALGSINT_BED",
    "TMP_FORBRUK_BED",
    "VAREKOST_BED",
    "GJELDENDE_DRIFTSK_KR",
    "DRIFTSKOST_KR",
    "FJOR_DRIFTSKOST_KR_T1",
    "NACEF_5",
    "SALGSINT",
    "FORBRUK",
    "TMP_NO_P4005",
    "TMP_AVPROS_ORGFORB",
    "ORGNR_N_1",
    "TMP_NO_OMSETN",
    "TMP_DRIFTSKOSTNAD_9010",
    "TMP_DRIFTSKOSTNAD_9910",
]

# Assuming `skjema` is your DataFrame and `felt_id_values` is your list of values
skjema = skjema[skjema["feltnavn"].isin(felt_id_values)]

In [ ]:
pivot_df = skjema.pivot_table(
    index=["id", "radnr", "lopenr"],
    columns="feltnavn",
    values="feltverdi",
    aggfunc="first",
)
pivot_df = pivot_df.reset_index()
pivot_df.columns = pivot_df.columns.str.lower()

pivot_df["year"] = year

pivot_df
# test = pivot_df['FJOR_OMSETN_KR_T1']
# test

In [ ]:
# Assuming your DataFrame is named pivot_df

# Create the 'foretak' DataFrame
foretak = pivot_df.loc[pivot_df["radnr"] == 0]

# Create the 'bedrift' DataFrame
bedrift = pivot_df.loc[pivot_df["radnr"] > 0]

In [ ]:
pd.set_option("display.max_columns", None)
# Assuming 'foretak' is your existing DataFrame

selected_columns = [
    "id",
    "lopenr",
    "forbruk",
    "nacef_5",
    "orgnr_n_1",
    "salgsint",
    "tmp_driftskostnad_9010",
    "tmp_driftskostnad_9910",
    "tmp_no_omsetn",
    "tmp_no_p4005",
]

foretak = foretak[selected_columns]

# Assuming 'foretak' is your DataFrame
foretak.rename(columns={"tmp_no_omsetn": "foretak_omsetning"}, inplace=True)


foretak = foretak.fillna(0)

foretak[["tmp_driftskostnad_9010", "tmp_driftskostnad_9910"]] = foretak[
    ["tmp_driftskostnad_9010", "tmp_driftskostnad_9910"]
].apply(pd.to_numeric, errors="coerce")

foretak["foretak_driftskostnad"] = foretak[
    ["tmp_driftskostnad_9010", "tmp_driftskostnad_9910"]
].max(axis=1)

# Drop the specified columns
foretak.drop(["tmp_driftskostnad_9010", "tmp_driftskostnad_9910"], axis=1, inplace=True)

In [ ]:
# Assuming 'bedrift' is your DataFrame
columns_to_drop = [
    "forbruk",
    "nacef_5",
    "orgnr_n_1",
    "salgsint",
    "tmp_driftskostnad_9010",
    "tmp_driftskostnad_9910",
    "tmp_no_omsetn",
    "tmp_no_p4005",
]

bedrift.drop(columns_to_drop, axis=1, inplace=True)

In [ ]:
# Assuming 'bedrift' is your DataFrame
columns_to_fill = ["omsetn_kr", "driftskost_kr"]

# Convert columns to numeric, replacing non-convertible values with NaN
bedrift[columns_to_fill] = bedrift[columns_to_fill].apply(
    pd.to_numeric, errors="coerce"
)

# Fill NaN values with 0 for the specified columns
bedrift[columns_to_fill] = bedrift[columns_to_fill].fillna(0)


# hjelpe virksomheter
if_condition = bedrift["regtype"] == "04"

# If the condition is True, set 'omsetn_kr' equal to 'driftskost_kr'
bedrift.loc[if_condition, "omsetn_kr"] = bedrift.loc[if_condition, "driftskost_kr"]


# Group by 'id' and calculate the sum
grouped_bedrift = (
    bedrift.groupby("id")[["omsetn_kr", "driftskost_kr"]].sum().reset_index()
)

# Rename the columns
grouped_bedrift.rename(
    columns={"omsetn_kr": "tot_oms_fordelt", "driftskost_kr": "tot_driftskost_fordelt"},
    inplace=True,
)

# Merge the grouped DataFrame back to the original DataFrame based on 'id'
bedrift = pd.merge(bedrift, grouped_bedrift, on="id", how="left")

In [ ]:
# Assuming 'foretak' and 'bedrift' are your DataFrames
merged_df = pd.merge(foretak, bedrift, on=["id", "lopenr"], how="inner")
# merged_df.head()

In [ ]:
# Assuming 'merged_df' is your DataFrame

# Convert columns to numeric, replacing non-convertible values with NaN
merged_df["tot_oms_fordelt"] = pd.to_numeric(
    merged_df["tot_oms_fordelt"], errors="coerce"
)
merged_df["foretak_omsetning"] = pd.to_numeric(
    merged_df["foretak_omsetning"], errors="coerce"
)

# Calculate omsetning_percentage
merged_df["omsetning_percentage"] = (
    merged_df["tot_oms_fordelt"] / merged_df["foretak_omsetning"]
)

# Convert columns to numeric, replacing non-convertible values with NaN
merged_df["tot_driftskost_fordelt"] = pd.to_numeric(
    merged_df["tot_driftskost_fordelt"], errors="coerce"
)
merged_df["foretak_driftskostnad"] = pd.to_numeric(
    merged_df["foretak_driftskostnad"], errors="coerce"
)

# Calculate driftskostnader_percentage
merged_df["driftskostnader_percentage"] = (
    merged_df["tot_driftskost_fordelt"] / merged_df["foretak_driftskostnad"]
)

merged_df["driftskostnader_percentage"] = (
    merged_df["tot_driftskost_fordelt"] / merged_df["foretak_driftskostnad"]
).round(4)

# Fill NaN with a specific value (e.g., 0)
merged_df["driftskostnader_percentage"].fillna(0, inplace=True)
merged_df["omsetning_percentage"].fillna(0, inplace=True)

In [ ]:
# merged_df.shape

In [ ]:
# Assuming 'merged_df' is your DataFrame

# Create the 'Good' DataFrame
good_temp_df = merged_df[
    (merged_df["omsetning_percentage"] >= limit)
    & (merged_df["driftskostnader_percentage"] >= limit)
]

# Create 'bedrift_count' and 'distribution_count'
good_temp_df["bedrift_count"] = good_temp_df.groupby("orgnr_n_1")[
    "orgnr_n_1"
].transform("count")
good_temp_df["distribution_count"] = good_temp_df.groupby("orgnr_n_1")[
    "omsetn_kr"
].transform(lambda x: (x > 0).sum())

# Create 'bad_temp' DataFrame based on conditions
bad_temp = good_temp_df[
    (good_temp_df["bedrift_count"] > 5) & (good_temp_df["distribution_count"] <= 2)
]

# Create 'good_df' by excluding rows from 'bad_temp'
good_df = (
    pd.merge(good_temp_df, bad_temp, how="outer", indicator=True)
    .query('_merge == "left_only"')
    .drop("_merge", axis=1)
)


good_df["oms_share"] = good_df["omsetn_kr"] / good_df["tot_oms_fordelt"].round(5)

# Round the values to whole numbers before assigning to the new columns
good_df["new_oms"] = (
    (good_df["oms_share"] * good_df["foretak_omsetning"]).round(0).astype(int)
)

good_df["oms_share"] = good_df["new_oms"] / good_df["tot_oms_fordelt"].round(5)


# Create the 'Mixed' DataFrame
onlygoodoms = merged_df[
    (
        (merged_df["omsetning_percentage"] > limit)
        & (merged_df["driftskostnader_percentage"] <= limit)
    )
]

onlygooddriftskostnader = merged_df[
    (
        (merged_df["driftskostnader_percentage"] > limit)
        & (merged_df["omsetning_percentage"] <= limit)
    )
]

# Create the 'Bad' DataFrame
bad_df = merged_df[
    (merged_df["omsetning_percentage"] <= limit)
    & (merged_df["driftskostnader_percentage"] <= limit)
]
bad_df = pd.concat([bad_df, bad_temp]).drop_duplicates(keep=False)
bad_df = pd.concat([bad_df, onlygooddriftskostnader]).drop_duplicates(keep=False)


del onlygooddriftskostnader

In [ ]:
merged_df.shape

In [ ]:
good_df.shape

In [ ]:
onlygoodoms.shape

In [ ]:
bad_df.shape

In [ ]:
good_df.head()

good distribution delivered (both oms and dkost): 83.33%
onlygoodoms: 11.96%
onlygooddriftskostnader: 0.693%
bad (bad distribution of both oms and dkost) : 4.02% 

# Step 1. 

Fill all bad df with NAN. For onlygoodoms use profit margin to impute driftskostnader.

In [ ]:
# Fill bad_df with NaN
bad_df["omsetn_kr"] = np.nan
bad_df["driftskost_kr"] = np.nan

bad_df["new_oms"] = bad_df["omsetn_kr"]

good_df["new_oms"] = good_df["omsetn_kr"]

# Correct driftskostnader for onlygoodoms

onlygoodoms["oms_share"] = onlygoodoms["omsetn_kr"] / onlygoodoms[
    "tot_oms_fordelt"
].round(5)

# Round the values to whole numbers before assigning to the new columns
onlygoodoms["new_oms"] = (
    (onlygoodoms["oms_share"] * onlygoodoms["foretak_omsetning"]).round(0).astype(int)
)

onlygoodoms["margin"] = (
    onlygoodoms["foretak_driftskostnad"] / onlygoodoms["foretak_omsetning"]
)

onlygoodoms["old_drkost"] = onlygoodoms["driftskost_kr"]

onlygoodoms["new_drkost"] = onlygoodoms["margin"] * onlygoodoms["new_oms"]

onlygoodoms["driftskost_kr"] = onlygoodoms["new_drkost"]

 

# Step 2: 

merge bad dfs back into good_df

In [ ]:
# Assuming 'onlygoodoms','bad_df', 'good_df', and 'onlygooddriftskostnader' are your DataFrames
# Merge all DataFrames into 'imputer' in a single step
merged_df = pd.concat([onlygoodoms, bad_df, good_df], ignore_index=True)

In [ ]:
# Assuming 'imputer' is your DataFrame and you want to select specific columns
selected_columns = [
    "id",
    "lopenr",
    "forbruk",
    "nacef_5",
    "orgnr_n_1",
    "salgsint",
    "foretak_omsetning",
    "tmp_no_p4005",
    "foretak_driftskostnad",
    "radnr",
    "gjeldende_bdr_syss",
    "fjor_driftskost_kr_t1",
    "fjor_lonn_kr_t1",
    "fjor_syssel_t1",
    "gjeldende_lonn_kr",
    "new_oms",
    "b_kommunenr",
]

# Create a new DataFrame with only the selected columns
imputer = merged_df[selected_columns].copy()

merged_df["n4"] = merged_df["nacef_5"].str[:5]

# merged_df.head(100)

# Assuming 'df' is your DataFrame
# nan_count = imputer['new_oms'].isna().sum()
# print("Number of NaN values in 'new_oms':", nan_count)

merged_df["new_oms"].value_counts()

# step 3 . 

Feature engineering:

- delta foretak oms
- delta kommune population
- delta kommune income
- change in employees
- consumer confidence index
- inflation
- gdp growth
- past data
- past years sales * foretak increase
- past years sales * kommune pop
- past years sales * kommune income

## Kommune belfolkning delta

In [ ]:
# kommune_pop_aar = kommune_pop.fetch_population_data(year)
# kommune_pop_fjor = kommune_pop.fetch_population_data(fjor)
kommune_befolk = kommune_pop.befolkning_behandling(year, fjor)

kommune_befolk.head()

## kommune inntekt delta

In [ ]:
# kommune_innt_aar = kommune_inntekt.fetch_inntekt_data(year)
# kommune_innt_fjor = kommune_inntekt.fetch_inntekt_data(fjor)
kommune_inn = kommune_inntekt.inntekt_behandling(year, fjor)
kommune_inn.head()

## Inflasjon data

In [ ]:
# kpi_total_data = kpi.fetch_kpi_data(year)
# kpi_hovedgruppe_data = kpi.fetch_hovedgruppe_data(year)
# kpi_gruppe_data = kpi.fetch_gruppe_data(year)
# subgruppe1_data = kpi.fetch_subgruppe1_data(year)

kpi_df = kpi.process_kpi_data(year)

# kpi_df = pd.concat([kpi_total_data, kpi_hovedgruppe_data, kpi_gruppe_data, subgruppe1_data], ignore_index=True)
# # Modify the DataFrame to keep rows where 'konsumgrp' is 45, 46, or 47
# kpi_df = kpi_df[kpi_df['konsumgrp'].str[:2].isin(["45", "46", "47"])]

# kpi_df = kpi_df.sort_values(by='konsumgrp')

# kpi_df['value'] = kpi_df['value'] / 100

# kpi_df.rename(columns={"value": "inflation_rate"}, inplace=True)

# # pd.set_option('display.max_rows', 100)

# kpi_df['n4'] = kpi_df[konsumgrp]

kpi_df.head()

In [ ]:
# Convert string columns to numeric
merged_df["gjeldende_bdr_syss"] = pd.to_numeric(
    merged_df["gjeldende_bdr_syss"], errors="coerce"
)
merged_df["fjor_syssel_t1"] = pd.to_numeric(
    merged_df["fjor_syssel_t1"], errors="coerce"
)

# Perform division after conversion
merged_df["emp_delta"] = merged_df["gjeldende_bdr_syss"] / merged_df["fjor_syssel_t1"]

In [ ]:
merged_df.head()

# Merge files with different features

keys: b_kommunenr, fjor_omsetn_kr_t1

inflation_rate_oms

In [ ]:
imputable_df = merged_df.copy()


imputable_df = imputable_df.drop_duplicates(subset=["v_orgnr"])

# imputable_df['n4'] =  imputable_df['nacef_5'].str[:5]
imputable_df["n4"] = imputable_df["tmp_sn2007_5"].str[:5]

imputable_df = pd.merge(imputable_df, kommune_befolk, on="b_kommunenr", how="left")
imputable_df = pd.merge(imputable_df, kommune_inn, on="b_kommunenr", how="left")
imputable_df = pd.merge(imputable_df, kpi_df, on="n4", how="left")

# Ensure columns are numeric
imputable_df["fjor_omsetn_kr_t1"] = pd.to_numeric(
    imputable_df["fjor_omsetn_kr_t1"], errors="coerce"
)
imputable_df["inflation_rate"] = pd.to_numeric(
    imputable_df["inflation_rate"], errors="coerce"
)
imputable_df["befolkning_delta"] = pd.to_numeric(
    imputable_df["befolkning_delta"], errors="coerce"
)
imputable_df["emp_delta"] = pd.to_numeric(imputable_df["emp_delta"], errors="coerce")
imputable_df["inntekt_delta"] = pd.to_numeric(
    imputable_df["inntekt_delta"], errors="coerce"
)

general_inflation_rate = imputable_df.loc[
    imputable_df["n4"] == "47.78", "inflation_rate"
].values[0]
imputable_df["inflation_rate"] = imputable_df["inflation_rate"].fillna(
    general_inflation_rate
)

imputable_df["inflation_rate_oms"] = (
    imputable_df["fjor_omsetn_kr_t1"] * imputable_df["inflation_rate"]
)
imputable_df["befolkning_delta_oms"] = (
    imputable_df["fjor_omsetn_kr_t1"] * imputable_df["befolkning_delta"]
)
imputable_df["emp_delta_oms"] = (
    imputable_df["fjor_omsetn_kr_t1"] * imputable_df["emp_delta"]
)
imputable_df["inntekt_delta_oms"] = (
    imputable_df["fjor_omsetn_kr_t1"] * imputable_df["inntekt_delta"]
)

# imputable_df['inflation_rate_oms'] = imputable_df['inflation_rate_oms'].round(0).astype(int)
# imputable_df['befolkning_delta_oms'] = imputable_df['befolkning_delta_oms'].round(0).astype(int)
# imputable_df['emp_delta_oms'] = imputable_df['emp_delta_oms'].round(0).astype(int)
# imputable_df['inntekt_delta_oms'] = imputable_df['inntekt_delta_oms'].round(0).astype(int)

# Treat Nan for inflation_rate_oms
imputable_df["inflation_rate_oms"].replace([np.inf, -np.inf], np.nan, inplace=True)
group_means = imputable_df.groupby("nacef_5")["inflation_rate_oms"].transform("mean")
# Step 3: Fill NaN values in 'inflation_rate_oms' with the corresponding group's mean
imputable_df["inflation_rate_oms"].fillna(group_means, inplace=True)


categories_to_impute = [
    "emp_delta_oms",
    "befolkning_delta_oms",
    "inntekt_delta_oms",
    "inflation_rate_oms",
]

# Identify rows where 'b_sysselsetting_syss' is equal to 0
rows_to_impute = imputable_df["b_sysselsetting_syss"] == 0

# Replace NaN values with 0 for the identified rows and specified categories
imputable_df.loc[rows_to_impute, categories_to_impute] = imputable_df.loc[
    rows_to_impute, categories_to_impute
].fillna(0)


# Group by 'tmp_sn2007_5' and calculate the average 'emp_delta_oms'
average_foretak_oms_pr_naring = imputable_df.groupby("tmp_sn2007_5")[
    "foretak_omsetning"
].mean()

# Create a new column 'average_emp_delt_oms_pr_naring' and assign the calculated averages to it
imputable_df["average_emp_delt_oms_pr_naring"] = imputable_df["nacef_5"].map(
    average_foretak_oms_pr_naring
)
imputable_df["average_emp_delt_oms_pr_naring"] = (
    imputable_df["average_emp_delt_oms_pr_naring"].round(0).astype(int)
)

knn_df = imputable_df[
    [
        "average_emp_delt_oms_pr_naring",
        "emp_delta_oms",
        "befolkning_delta_oms",
        "inflation_rate_oms",
        "inntekt_delta_oms",
        "b_sysselsetting_syss",
        "v_orgnr",
    ]
]
knn_df = knn_df.replace([np.inf, -np.inf], np.nan)


# imputable_df.head(100)

In [ ]:
knn_df.head()

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Create a copy of your DataFrame
imputable_df_copy = knn_df.copy()

# Define the columns for numerical features
numerical_features = [
    "average_emp_delt_oms_pr_naring",
    "emp_delta_oms",
    "befolkning_delta_oms",
    "inflation_rate_oms",
    "inntekt_delta_oms",
    "b_sysselsetting_syss",
]

numerical_features = [col for col in numerical_features if col != "v_orgnr"]


# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[("num", StandardScaler(), numerical_features)],
    remainder="passthrough",  # Keep non-numerical columns unchanged
)

knn_df["v_orgnr"] = knn_df["v_orgnr"].astype(str)

# Create KNN imputer
knn_imputer = KNNImputer(n_neighbors=3)

# Create imputer pipeline
imputer_pipeline = Pipeline([("preprocessor", preprocessor), ("imputer", knn_imputer)])

# Fit and transform the copy of your DataFrame
imputed_values = imputer_pipeline.fit_transform(imputable_df_copy)


# Convert the imputed values back to a DataFrame
knn_df = pd.DataFrame(imputed_values, columns=imputable_df_copy.columns)


# Update only the NaN values in the original DataFrame with the imputed values
knn_df.update(knn_df, overwrite=False)

# Inverse transform the scaled numerical features
scaled_features = knn_df[numerical_features].values
inverse_scaled_features = preprocessor.named_transformers_["num"].inverse_transform(
    scaled_features
)
knn_df[numerical_features] = inverse_scaled_features

In [ ]:
knn_df["v_orgnr"] = knn_df["v_orgnr"].round(0).astype(int)
knn_df["v_orgnr"] = knn_df["v_orgnr"].astype(object)
columns_to_drop = ["average_emp_delt_oms_pr_naring", "b_sysselsetting_syss"]
knn_df.drop(columns=columns_to_drop, axis=1, inplace=True)
knn_df.dtypes

In [ ]:
knn_df.head()

In [ ]:
columns_to_drop = [
    "emp_delta_oms",
    "befolkning_delta_oms",
    "inflation_rate_oms",
    "inntekt_delta_oms",
]

imputable_df.drop(columns=columns_to_drop, axis=1, inplace=True)

In [ ]:
duplicate_count_imputable = imputable_df["v_orgnr"].duplicated().sum()
print("Number of duplicates in imputable_df:", duplicate_count_imputable)

In [ ]:
imputable_df.dtypes

In [ ]:
# Convert 'v_orgnr' column to strings
knn_df["v_orgnr"] = knn_df["v_orgnr"].astype(str)
imputable_df["v_orgnr"] = imputable_df["v_orgnr"].astype(str)

# Strip 'v_orgnr' column in both knn_df and imputable_df
knn_df["v_orgnr"] = knn_df["v_orgnr"].str.strip()
imputable_df["v_orgnr"] = imputable_df["v_orgnr"].str.strip()

imputable_df = pd.merge(imputable_df, knn_df, how="inner", on="v_orgnr")

imputable_df.head()

In [ ]:
duplicate_v_orgnr_imputable = imputable_df["v_orgnr"].duplicated().any()
duplicate_v_orgnr_knn = knn_df["v_orgnr"].duplicated().any()

print("Duplicate v_orgnr in imputable_df:", duplicate_v_orgnr_imputable)
print("Duplicate v_orgnr in knn_df:", duplicate_v_orgnr_knn)

In [ ]:
# Sample data with NaN values
columns_for_imputation = [
    "new_oms",
    "nacef_5",
    "inntekt_delta_oms",
    "emp_delta_oms",
    "befolkning_delta_oms",
    "inflation_rate_oms",
    "v_orgnr",
    "gjeldende_bdr_syss",
]

#####################################
# DIFFERENT FILTERING
################################


# Assuming 'reg_type' is the column in imputable_df
# imputable_df_filtered = imputable_df[imputable_df['regtype'].isin(['01', '02', '03'])]
imputable_df_filtered = imputable_df[~imputable_df["regtype"].isin(["04", "11"])]


#####################################
# DIFFERENT FILTERING
################################


filtered_imputation_df = imputable_df_filtered[columns_for_imputation]
filtered_imputation_df.replace([np.inf, -np.inf], np.nan, inplace=True)


# # Filter for rows with at least one non-NaN value in the specified columns
# cleaned_imputation_df = filtered_imputation_df[filtered_imputation_df[['inntekt_delta_oms', 'emp_delta_oms', 'befolkning_delta_oms', 'inflation_rate_oms']].notna().any(axis=1)]

In [ ]:
# Select columns needed for the model (excluding 'new_oms')
columns_for_imputation = filtered_imputation_df.columns.tolist()
columns_for_imputation.remove("new_oms")

# Filter for rows where all columns (except 'new_oms') have no NaN values
cleaned_imputation_df = filtered_imputation_df.dropna(
    subset=columns_for_imputation, how="any"
)

# Filter for rows where at least one column (excluding 'new_oms') has NaN values
nn_df = filtered_imputation_df[
    filtered_imputation_df[columns_for_imputation].isna().any(axis=1)
]

In [ ]:
cleaned_imputation_df["inflation_rate_oms"] = (
    cleaned_imputation_df["inflation_rate_oms"].round(0).astype(int)
)
cleaned_imputation_df["befolkning_delta_oms"] = (
    cleaned_imputation_df["befolkning_delta_oms"].round(0).astype(int)
)
cleaned_imputation_df["emp_delta_oms"] = (
    cleaned_imputation_df["emp_delta_oms"].round(0).astype(int)
)
cleaned_imputation_df["inntekt_delta_oms"] = (
    cleaned_imputation_df["inntekt_delta_oms"].round(0).astype(int)
)

In [ ]:
nan_count = cleaned_imputation_df["new_oms"].isna().sum()
print(f"The number of NaN values in the 'new_oms' column is: {nan_count}")

In [ ]:
# Assuming you have cleaned_imputation_df and filtered_imputation_df

# Get the indices of rows present in filtered_imputation_df
filtered_indices = filtered_imputation_df.index.tolist()

# Filter rows from filtered_imputation_df that are not present in cleaned_imputation_df (based on index)
nn_df = filtered_imputation_df[
    ~filtered_imputation_df.index.isin(cleaned_imputation_df.index)
]

# Alternative (less efficient for large DataFrames):
# nn_df = filtered_imputation_df[~filtered_imputation_df.isin(cleaned_imputation_df).any(axis=1)]

In [ ]:
nn_df.head()

In [ ]:
nn_df.dtypes

In [ ]:
nan_rows = nn_df[nn_df["emp_delta_oms"].isna()]

# Display the filtered DataFrame
print("\nRows where 'emp_delta_oms' is NaN:")
nan_rows.head()

## Create test set

In [ ]:
cleaned_imputation_df.head()

test_df = cleaned_imputation_df.copy()

test_df = test_df.dropna()

test_df["real_oms"] = test_df["new_oms"]

# Define the proportion of NaNs (adjust as needed)
nan_proportion = 0.06  # 6% of values will be NaN - approx what we see in reality

# Get the number of rows in the DataFrame
num_rows = test_df.shape[0]

# Calculate the number of rows to assign NaN
num_nan = int(num_rows * nan_proportion)

# Randomly sample indices for NaN assignment (without replacement)
nan_indices = test_df.sample(
    num_nan, random_state=42
).index  # Set random_state for reproducibility

test_df.loc[nan_indices, "new_oms"] = np.nan

test_df.dtypes

# Feature selection

In [ ]:
# from sklearn.feature_selection import SelectKBest, f_regression
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split

# # Load your test data (assuming it's in a DataFrame named test_df)

# # Separate features (excluding v_orgnr) and target variable
# features = test_df.drop(['v_orgnr', 'real_oms', 'new_oms'], axis=1)
# target = test_df['new_oms']

# # Handle NaN values by dropping rows (consider alternative imputation if needed)
# features_dropna = features.dropna()
# target_dropna = target.loc[features_dropna.index]  # Align target with dropped rows

# # Split data into training and testing sets (optional, for model evaluation)
# X_train, X_test, y_train, y_test = train_test_split(features_dropna, target_dropna, test_size=0.2, random_state=42)

# # Feature selection with SelectKBest (f_regression for correlation-like measure)
# selector = SelectKBest(f_regression, k=2)  # Select top 5 features
# selector.fit(X_train, y_train)

# # Print the selected features
# print("Selected features:")
# print(features_dropna.columns[selector.get_support(indices=True)])

# # Train a Random Forest model
# model_rf = RandomForestRegressor()
# model_rf.fit(X_train, y_train)

# # Get feature importances from the Random Forest model
# importances = model_rf.feature_importances_

# # Print top 5 features based on Random Forest importance
# print("\nTop 5 features by Random Forest importance:")
# print(features_dropna.columns[importances.argsort()[-2:]])

# # ... (Optional: Use the trained model for predictions on new data)

# Interative Imputer + testing

In [ ]:
# Works but no evaluation

import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import mean_squared_error  # for evaluation (optional)
from sklearn.metrics import mean_absolute_error  # for evaluation (optional)
from sklearn.metrics import median_absolute_error  # for evaluation (optional)

# Assuming cleaned_imputation_df is your existing dataframe

# Make a copy to avoid modifying the original data
X = cleaned_imputation_df.copy()

# Separate features (excluding v_orgnr) and target variable
features = X.drop(["v_orgnr", "new_oms"], axis=1)
target = X["new_oms"]

# Impute missing values in features (if applicable)
impute_it = IterativeImputer()
impute_it.fit(features)
features_transformed = impute_it.transform(features)
features_df = pd.DataFrame(
    features_transformed, columns=features.columns
)  # Convert to DataFrame

# Train your model here using features_df and target
# (Replace this with your specific model training code)

# Keep a copy of the original DataFrame for merging later
original_df = X.copy()  # This maintains v_orgnr for merging

# Create a DataFrame with imputed new_oms values (optional)
imputed_df = features_df.copy()
imputed_df["v_orgnr"] = X["v_orgnr"]  # Add v_orgnr from the original DataFrame

In [ ]:
# Test imputation:

# Make a copy to avoid modifying the original data
X = test_df.copy()

# Separate features (excluding v_orgnr) and target variable
features = X.drop(
    [
        "v_orgnr",
        "real_oms",
        "befolkning_delta_oms",
        "inflation_rate_oms",
        "emp_delta_oms",
    ],
    axis=1,
)
target = X["new_oms"]

# Impute missing values in features (if applicable)
impute_it = IterativeImputer()
impute_it.fit(features)
features_transformed = impute_it.transform(features)
features_df = pd.DataFrame(
    features_transformed, columns=features.columns
)  # Convert to DataFrame

# Train your model here using features_df and target
# (Replace this with your specific model training code)

# Keep a copy of the original DataFrame for merging later
original_test_df = X.copy()  # This maintains v_orgnr for merging

# Create a DataFrame with imputed new_oms values (optional)
imputed_test_df = features_df.copy()
imputed_test_df["v_orgnr"] = X["v_orgnr"]  # Add v_orgnr from the original DataFrame

In [ ]:
evaluate_df = pd.merge(
    test_df, imputed_test_df[["v_orgnr", "new_oms"]], how="left", on="v_orgnr"
)
evaluate_df = evaluate_df[
    evaluate_df["new_oms_x"].isna()
]  # Filter rows where 'new_oms_x' is NaN

evaluate_df.rename(columns={"new_oms_y": "predicted_oms"}, inplace=True)
check = evaluate_df[evaluate_df["predicted_oms"].isna()]
evaluate_df = evaluate_df[evaluate_df["predicted_oms"].notna()]

evaluate_df.head()

In [ ]:
nan_count = evaluate_df["predicted_oms"].isna().sum()
print(f"The number of NaN values in the 'predicted_oms' column is: {nan_count}")

In [ ]:
nan_count = test_df["new_oms"].isna().sum()
print(f"The number of NaN values in the 'new_oms' column is: {nan_count}")

In [ ]:
check = evaluate_df[
    evaluate_df["predicted_oms"].isna()
]  # Filter rows where 'new_oms_x' is NaN

check["inntekt_delta_oms"] = pd.to_numeric(
    check["inntekt_delta_oms"], errors="coerce"
)  # Attempt numeric conversion


check.head(100)

In [ ]:
# Calculate metrics
mse = mean_squared_error(evaluate_df["real_oms"], evaluate_df["predicted_oms"])
mae = mean_absolute_error(evaluate_df["real_oms"], evaluate_df["predicted_oms"])
medae = median_absolute_error(evaluate_df["real_oms"], evaluate_df["predicted_oms"])
# (Optional) Percentage Error (PE) calculation
# You might need to adjust the logic based on your data
# evaluate_df['pe'] = (abs(evaluate_df['predicted_oms'] - evaluate_df['real_oms']) / evaluate_df['real_oms']) * 100

# Print the results
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")
# (Optional) Print PE if calculated
# print(f"Percentage Error (PE): {evaluate_df['pe'].mean()}")

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot of predicted vs. real OMS
plt.scatter(evaluate_df["real_oms"], evaluate_df["predicted_oms"])
plt.xlabel("Real OMS")
plt.ylabel("Predicted OMS")
plt.title("Predicted vs. Real OMS")
plt.show()

# Optional: Residual plot
plt.scatter(
    evaluate_df["real_oms"], evaluate_df["predicted_oms"] - evaluate_df["real_oms"]
)
plt.xlabel("Real OMS")
plt.ylabel("Residuals (Predicted - Real)")
plt.title("Residual Plot")
plt.show()

### With Every variable

- Mean Squared Error (MSE): 510766522350.0012
- Mean Absolute Error (MAE): 108769.08051029009
- Median Absolute Error (MedAE): 24932.0

## With Just employee delta

- Mean Squared Error (MSE): 510144521616.9896
- Mean Absolute Error (MAE): 108323.35968378294
- Median Absolute Error (MedAE): 25709.0

## With just inflation data

- Mean Squared Error (MSE): 510672841283.873
- Mean Absolute Error (MAE): 108804.11145815151
- Median Absolute Error (MedAE): 25081.0

# KNN Imputer

## Make KNN test set and main dataset for imputing/fixing

In [ ]:
# test_knn_df = test_df.copy()
# # final_df = pd.merge(cleaned_imputation_df, df[['v_orgnr', 'new_oms']], how='left', on='v_orgnr')
# test_knn_df = pd.merge(test_knn_df, imputable_df[['v_orgnr', 'nacef_5', 'b_kommunenr', 'b_sysselsetting_syss']], how='left', on='v_orgnr')

In [ ]:
hoved_df = pd.merge(
    cleaned_imputation_df,
    imputable_df[["v_orgnr", "tmp_sn2007_5", "b_kommunenr", "b_sysselsetting_syss"]],
    how="left",
    on="v_orgnr",
)

In [ ]:
hoved_df["b_sysselsetting_syss"] = hoved_df["b_sysselsetting_syss"].fillna(0)

In [ ]:
# Count NaN values in each column
nan_counts = hoved_df.isna().sum()

# Filter columns with NaN values
columns_with_nan = nan_counts[nan_counts > 0]

# Print columns with NaN values and their respective counts
print("Columns with NaN values:")
print(columns_with_nan)

In [ ]:
nan_count = hoved_df["new_oms"].isna().sum()
print(f"The number of NaN values in the 'new_oms' column is: {nan_count}")

In [ ]:
hoved_df.dtypes

In [ ]:
# test_knn_df['b_sysselsetting_syss'] = test_knn_df['b_sysselsetting_syss'].round(0).astype(int)
# Convert the column to numeric, forcing errors to NaN
hoved_df["b_sysselsetting_syss"] = pd.to_numeric(
    hoved_df["b_sysselsetting_syss"], errors="coerce"
)

# Now you can round the values and convert them to integers
hoved_df["b_sysselsetting_syss"] = (
    hoved_df["b_sysselsetting_syss"].round(0).astype("Int64")
)  # Using 'Int64' to handle NaN values


# knn_imputation_df = pd.merge(cleaned_imputation_df, imputable_df[['v_orgnr', 'nacef_5', 'b_kommunenr', 'b_sysselsetting_syss']], how='left', on='v_orgnr')
# knn_imputation_df['b_sysselsetting_syss'] = knn_imputation_df['b_sysselsetting_syss'].round(0).astype(int)

In [ ]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

# Assuming cleaned_imputation_df is your existing dataframe

# Because its imputing, you can include 'new_oms'

# Make a copy to avoid modifying the original data

# Use cleaned_imputation_df for real imputation
# Use test_knn_df for test imputation

# X = test_knn_df.copy()
X = hoved_df.copy()

drop_variables = ["v_orgnr", "real_oms"]

# Separate features (excluding v_orgnr) and target variable
features = X.drop([col for col in drop_variables if col in X.columns], axis=1)
target = X["new_oms"]

# Define categorical features for target encoding
categorical_features = ["nacef_5", "b_kommunenr"]

# Create a dictionary to store average target (new_oms) for each category
target_encoding_dict = {}
for cat_feature in categorical_features:
    target_encoding_dict[cat_feature] = features.groupby(cat_feature)["new_oms"].mean()


# Function to apply target encoding to a single feature
def apply_target_encoding(df, feature, encoding_dict):
    df[feature] = df[feature].replace(encoding_dict[feature].to_dict())
    return df


# Apply target encoding to each categorical feature
for cat_feature in categorical_features:
    features = apply_target_encoding(features.copy(), cat_feature, target_encoding_dict)

# # Handle missing values in numerical features (optional)
# # You can replace this with your preferred imputation method
# imputer = SimpleImputer(strategy='mean')
# features = imputer.fit_transform(features)

# Feature scaling (standardization in this example) on numerical features only
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# features_df = pd.DataFrame(scaled_features, columns=features.columns)  # Use original feature names

# # Convert scaled features back to a DataFrame
# scaled_df = pd.DataFrame(scaled_features, columns=features.columns)

# ... (Rest of your code for processing other features)

# Combine encoded categorical features with scaled numerical features
# all_features = pd.concat([scaled_df, features[list(set(features.columns) - set(categorical_features))]], axis=1)


# Impute missing values in features using KNN imputer
imputer = KNNImputer(n_neighbors=3)
imputer.fit(scaled_features)
features_transformed = imputer.transform(scaled_features)

# Create a DataFrame with imputed features (including original column names)
imputed_features = pd.DataFrame(features_transformed, columns=features.columns)

# Inverse scaling on imputed features to get original scale back
imputed_features = pd.DataFrame(
    scaler.inverse_transform(imputed_features), columns=features.columns
)

# Train your model here using features_df and target
# (Replace this with your specific model training code)

# Combine imputed features with target variable (optional)
imputed_knn_df = imputed_features.copy()
# imputed_df['new_oms'] = target  # Assuming you want the target variable

# Combine with organization number
imputed_knn_df["v_orgnr"] = X["v_orgnr"]

In [ ]:
nan_count = imputed_knn_df["new_oms"].isna().sum()
print(f"The number of NaN values in the 'new_oms' column is: {nan_count}")

original_nan_count = hoved_df["new_oms"].isna().sum()
print(
    f"The number of NaN values originally in the 'new_oms' column is: {original_nan_count}"
)

shape = imputed_knn_df.shape
shape_original = hoved_df.shape

print(f"The shape now is: {shape}")
print(f"The shape originally was: {shape_original}")

In [ ]:
# evaluate_knn_df = pd.merge(test_knn_df, imputed_knn_df[['v_orgnr', 'new_oms']], how='left', on='v_orgnr')
# evaluate_knn_df = evaluate_knn_df[evaluate_knn_df['new_oms_x'].isna()]  # Filter rows where 'new_oms_x' is NaN

# evaluate_knn_df.rename(columns={"new_oms_y": "predicted_oms"}, inplace=True)
# check = evaluate_knn_df[evaluate_knn_df['predicted_oms'].isna()]
# evaluate_knn_df = evaluate_knn_df[evaluate_knn_df['predicted_oms'].notna()]

# evaluate_knn_df.head()

In [ ]:
# nan_count = evaluate_knn_df['predicted_oms'].isna().sum()
# print(f"The number of NaN values in the 'predicted_oms' column is: {nan_count}")

In [ ]:
# nan_count = test_knn_df['new_oms'].isna().sum()
# print(f"The number of NaN values in the 'new_oms' column is: {nan_count}")

In [ ]:
# # Calculate metrics
# mse = mean_squared_error(evaluate_knn_df['real_oms'], evaluate_knn_df['predicted_oms'])
# mae = mean_absolute_error(evaluate_knn_df['real_oms'], evaluate_knn_df['predicted_oms'])
# medae = median_absolute_error(evaluate_knn_df['real_oms'], evaluate_knn_df['predicted_oms'])
# # (Optional) Percentage Error (PE) calculation
# # You might need to adjust the logic based on your data
# # evaluate_df['pe'] = (abs(evaluate_df['predicted_oms'] - evaluate_df['real_oms']) / evaluate_df['real_oms']) * 100

# # Print the results
# print(f"Mean Squared Error (MSE): {mse}")
# print(f"Mean Absolute Error (MAE): {mae}")
# print(f"Median Absolute Error (MedAE): {medae}")
# # (Optional) Print PE if calculated
# # print(f"Percentage Error (PE): {evaluate_df['pe'].mean()}")

In [ ]:
# import matplotlib.pyplot as plt

# # Scatter plot of predicted vs. real OMS
# plt.scatter(evaluate_knn_df['real_oms'], evaluate_knn_df['predicted_oms'])
# plt.xlabel('Real Sales Turnover')
# plt.ylabel('Predicted Sales Turnover')
# plt.title('Predicted vs. Real Sales Turnover')
# plt.show()

# # Optional: Residual plot
# plt.scatter(evaluate_knn_df['real_oms'], evaluate_knn_df['predicted_oms'] - evaluate_knn_df['real_oms'])
# plt.xlabel('Real Sales Turnvoer')
# plt.ylabel('Residuals (Predicted - Real)')
# plt.title('Residual Plot')
# plt.show()

# Neural Networking

In [ ]:
# New model

In [ ]:
# # Check which columns contain NaN values
# nan_columns = hoved_df.columns[hoved_df.isna().any()].tolist()

# # Print the columns that contain NaN values
# print(f"Columns containing NaN values: {nan_columns}")

# nan_rows = hoved_df[hoved_df['tmp_sn2007_5'].isna() | hoved_df['b_kommunenr'].isna()]

# nan_rows.head()

# impute missing values:

import pandas as pd

# Assuming hoved_df is your DataFrame

# Replace any string 'NaN' with actual NaN for proper checking
hoved_df.replace("nan", np.nan, inplace=True)

# Step 1: Delete the row with v_orgnr = 111111111
hoved_df = hoved_df[hoved_df["v_orgnr"] != "111111111"]

# Step 2: Impute the missing values for the specified rows
impute_values = {
    "921757131": {"tmp_sn2007_5": "46.710", "b_kommunenr": "0301"},
    "998505461": {"tmp_sn2007_5": "32.990", "b_kommunenr": "4622"},
    "928019691": {"tmp_sn2007_5": "32.990", "b_kommunenr": "4621"},
    "928601196": {"tmp_sn2007_5": "62.020", "b_kommunenr": "1507"},
}

for v_orgnr, values in impute_values.items():
    if "tmp_sn2007_5" in values:
        hoved_df.loc[hoved_df["v_orgnr"] == v_orgnr, "tmp_sn2007_5"] = values[
            "tmp_sn2007_5"
        ]
    if "b_kommunenr" in values:
        hoved_df.loc[hoved_df["v_orgnr"] == v_orgnr, "b_kommunenr"] = values[
            "b_kommunenr"
        ]

# Ensure changes are applied and convert back to string
hoved_df = hoved_df.astype({"v_orgnr": str, "tmp_sn2007_5": str, "b_kommunenr": str})

# Verify the changes
print(hoved_df[hoved_df["v_orgnr"].isin(impute_values.keys())])

# Check for any remaining NaN values in 'tmp_sn2007_5' or 'b_kommunenr'
nan_rows = hoved_df[hoved_df["tmp_sn2007_5"].isna() | hoved_df["b_kommunenr"].isna()]
print(nan_rows.head())

In [ ]:
# Check for any remaining NaN values in 'tmp_sn2007_5' or 'b_kommunenr'
nan_rows = hoved_df[hoved_df["tmp_sn2007_5"].isna() | hoved_df["b_kommunenr"].isna()]
print(nan_rows.head())

In [ ]:
# nan_rows = hoved_df[hoved_df['tmp_sn2007_5'].isna() | hoved_df['b_kommunenr'].isna()]

# nan_rows.head()

# hoved_df.head()

negative_oms_rows = hoved_df[hoved_df["new_oms"] < 0]
negative_oms_rows.head()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    median_absolute_error,
    r2_score,
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Assuming hoved_df is already defined and loaded

# Separate features and target
drop_variables = ["v_orgnr", "nacef_5"]

# Filter the dataframe
# small_hoved_df = hoved_df[hoved_df['new_oms'] < 100000]
# large_hoved_df = hoved_df[hoved_df['new_oms'] > 100000]

# Calculate the 80th percentile of 'new_oms' column
top_percentile = hoved_df["new_oms"].quantile(0.8)

# Filter the dataframe
# top_80_df = hoved_df[hoved_df['new_oms'] >= top_percentile]
# bottom_80_df = hoved_df[hoved_df['new_oms'] < top_percentile]

# All
X = hoved_df.copy()

# Small
# X = small_hoved_df.copy()

# # large
# X = large_hoved_df.copy()

# # Largest percentile
# X = top_80_df.copy()

# Smallest Percentile
# X = bottom_80_df.copy()


features = X.drop([col for col in drop_variables if col in X.columns], axis=1)
target = X["new_oms"]

# Handle categorical features using target encoding
categorical_features = ["tmp_sn2007_5", "b_kommunenr"]
temp_df = features.copy()
temp_df["new_oms"] = target

target_encoding_dict = {}
for cat_feature in categorical_features:
    target_encoding_dict[cat_feature] = temp_df.groupby(cat_feature)["new_oms"].mean()


def apply_target_encoding(df, feature, encoding_dict):
    df[feature] = df[feature].replace(encoding_dict[feature].to_dict())
    return df


for cat_feature in categorical_features:
    features = apply_target_encoding(features.copy(), cat_feature, target_encoding_dict)

# Drop 'new_oms' from features
features = features.drop("new_oms", axis=1)

# Split the data into non-NaN and NaN parts
non_nan_mask = ~target.isna()
features_non_nan = features[non_nan_mask]
target_non_nan = target[non_nan_mask]
features_nan = features[~non_nan_mask]

# Feature scaling (standardization)
scaler = StandardScaler()
scaled_features_non_nan = scaler.fit_transform(features_non_nan)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    scaled_features_non_nan, target_non_nan, test_size=0.2, random_state=42
)

# Define the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="linear"))

# Compile the model
model.compile(optimizer="adam", loss="mean_absolute_error")

# Early stopping
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=150,
    batch_size=8,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stopping],
)

# Evaluate the model on the test set
y_pred = model.predict(X_test).flatten()

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)

# Calculate mean standard error ratio
mean_target = np.mean(y_test)
mae_ratio = mae / mean_target

# Print the results
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")
print(f"Mean Standard Error Ratio: {mae_ratio}")
print(f"R Squared: {r_squared}")

# Plot the training history
plt.plot(history.history["loss"], label="train loss")
plt.plot(history.history["val_loss"], label="validation loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training History")
plt.show()

# Scatter plot of predicted vs. real OMS
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
plt.xlabel("Real Sales Turnover")
plt.ylabel("Predicted Sales Turnover")
plt.title("Predicted vs. Real Sales Turnover")
plt.show()

# Optional: Residual plot
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred - y_test, alpha=0.3)
plt.hlines(0, y_test.min(), y_test.max(), colors="r", linestyles="dashed")
plt.xlabel("Real Sales Turnover")
plt.ylabel("Residuals (Predicted - Real)")
plt.title("Residual Plot")
plt.show()

# Scale the features for NaN rows
scaled_features_nan = scaler.transform(features_nan)

# Predict missing 'new_oms' values
predicted_nan = model.predict(scaled_features_nan).flatten()

# Impute the predicted values back into the original dataframe
X.loc[~non_nan_mask, "new_oms"] = predicted_nan

# Verify the imputation
print(X.head())
print(X[~non_nan_mask].head())  # Check the rows where NaN values were imputed

# XGBOOST

In [ ]:
hoved_df.shape

In [ ]:
hoved_df = hoved_df.dropna()
hoved_df.shape

In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import shap

# df = ml_df.copy()
# df = pre_ml.copy()
df = hoved_df.copy()
# df = test_uten2017.copy()

# df = df[np.isfinite(df["mengde"])]
# Replace infinite values with NaN
df = df.replace([np.inf, -np.inf], np.nan)


# Drop 'ident' column if not needed
df = df.drop(columns=["v_orgnr"])

# Convert object types to category
categorical_columns = ["tmp_sn2007_5", "b_kommunenr", "nacef_5"]
for col in categorical_columns:
    df[col] = df[col].astype("category")

# Apply log transformation to `verdi`
# df['verdi'] = np.log1p(df['verdi'])

# Define features and target variable
X = df.drop(columns=["new_oms"])
y = df["new_oms"]

# Define categorical and numerical features
categorical_features = ["tmp_sn2007_5", "b_kommunenr", "nacef_5"]
numerical_features = [
    "inntekt_delta_oms",
    "emp_delta_oms",
    "befolkning_delta_oms",
    "inflation_rate_oms",
    "gjeldende_bdr_syss",
]

# # Preprocessing pipeline
# preprocessor = ColumnTransformer(
#     transformers=[
#         ("num", "passthrough", numerical_features),  # No transformation for numerical features
#         ("cat", OneHotEncoder(categories="auto", handle_unknown="ignore"), categorical_features),  # One-hot encoding for categorical features
#     ]
# )

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        (
            "num",
            StandardScaler(),
            numerical_features,
        ),  # Apply standard scaling to numerical features
        (
            "cat",
            OneHotEncoder(categories="auto", handle_unknown="ignore"),
            categorical_features,
        ),  # One-hot encoding for categorical features
    ]
)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Fit the preprocessor on the training data
preprocessor.fit(X_train)

# # Transform the training and testing data
# X_train_transformed = preprocessor.transform(X_train)
# X_test_transformed = preprocessor.transform(X_test)

# Transform the training and testing data
X_train_transformed = preprocessor.transform(X_train).toarray()
X_test_transformed = preprocessor.transform(X_test).toarray()

# Define the model
# regressor = xgb.XGBRegressor(eval_metric='rmse', enable_categorical=True)
regressor = xgb.XGBRegressor(eval_metric="mae", enable_categorical=True)

# Train the model with learning history
eval_set = [(X_train_transformed, y_train), (X_test_transformed, y_test)]
regressor.fit(
    X_train_transformed,
    y_train,
    eval_set=eval_set,
    early_stopping_rounds=10,
    verbose=False,
)

# Evaluate the model
y_pred = regressor.predict(X_test_transformed)


mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)  # Calculate Mean Absolute Error
print("Mean Squared Error:", mse)
print("R-squared:", r_squared)
print("Mean Absolute Error:", mae)

# Cross-validation for more robust evaluation
cv_scores = cross_val_score(
    Pipeline([("preprocessor", preprocessor), ("regressor", regressor)]),
    X,
    y,
    cv=5,
    scoring="neg_mean_squared_error",
)
print("Cross-Validation Mean Squared Error:", -cv_scores.mean())

# Plot the learning history
results = regressor.evals_result()
# epochs = len(results["validation_0"]["rmse"])
epochs = len(results["validation_0"]["mae"])
x_axis = range(0, epochs)
plt.figure(figsize=(10, 5))
# plt.plot(x_axis, results["validation_0"]["rmse"], label="Train")
# plt.plot(x_axis, results["validation_1"]["rmse"], label="Test")
plt.plot(x_axis, results["validation_0"]["mae"], label="Train")
plt.plot(x_axis, results["validation_1"]["mae"], label="Test")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("RMSE")
plt.title("XGBoost Learning History")
plt.show()

# Plot Predicted vs. Actual Values
plt.figure(figsize=(10, 5))
plt.scatter(y_test, y_pred, alpha=0.3)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", lw=2)
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.title("Predicted vs. Actual Values")
plt.show()

# Plot Residuals
residuals = y_test - y_pred
plt.figure(figsize=(10, 5))
plt.scatter(y_test, residuals, alpha=0.3)
plt.hlines(0, y_test.min(), y_test.max(), colors="r", linestyles="dashed")
plt.xlabel("Actual")
plt.ylabel("Residuals")
plt.title("Residuals Plot")
plt.show()


# Tree textual representation
booster = regressor.get_booster()
with open("dump.raw.txt", "w") as f:
    f.write("\n".join(booster.get_dump()))
print(booster.get_dump()[0])  # Print the first tree

# SHAP values
explainer = shap.Explainer(regressor, X_train_transformed)
shap_values = explainer(X_test_transformed)

# Get feature names after one-hot encoding
feature_names = preprocessor.get_feature_names_out()

# Summary plot of SHAP values
shap.summary_plot(shap_values, X_test_transformed, feature_names=feature_names)

# Force plot for a single prediction (e.g., the first instance)
shap.initjs()
shap.force_plot(
    shap_values[0].base_values,
    shap_values[0].values,
    X_test_transformed[0],
    feature_names=feature_names,
)

# Find the correct index for the feature "verdi"
verdi_index = list(feature_names).index("num__inntekt_delta_oms")

# Dependence plot to show the effect of a single feature across the dataset
shap.dependence_plot(
    verdi_index, shap_values.values, X_test_transformed, feature_names=feature_names
)

In [ ]:
# Old model - works for dfs without NaN new_oms

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    median_absolute_error,
)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

# Assuming test_knn_df is your dataframe after merging imputable_df and test_df

# Make a copy to avoid modifying the original data

# Test data for evaluating = test_knn_df
# real data = hoved_df

# X = test_knn_df.copy()
X = hoved_df.copy()

drop_variables = ["v_orgnr", "nacef_5"]

# Separate features (excluding v_orgnr) and target variable
features = X.drop([col for col in drop_variables if col in X.columns], axis=1)
target = X["new_oms"]

# Handle categorical features using target encoding
categorical_features = ["tmp_sn2007_5", "b_kommunenr"]

# Create a temporary dataframe including the target to compute the mean
temp_df = features.copy()
temp_df["new_oms"] = target

target_encoding_dict = {}
for cat_feature in categorical_features:
    target_encoding_dict[cat_feature] = temp_df.groupby(cat_feature)["new_oms"].mean()


def apply_target_encoding(df, feature, encoding_dict):
    df[feature] = df[feature].replace(encoding_dict[feature].to_dict())
    return df


for cat_feature in categorical_features:
    features = apply_target_encoding(features.copy(), cat_feature, target_encoding_dict)

# Now drop 'new_oms' from features
features = features.drop("new_oms", axis=1)

# Drop rows where target 'new_oms' is NaN
non_nan_mask = ~target.isna()
features_non_nan = features[non_nan_mask]
target_non_nan = target[non_nan_mask]

# Feature scaling (standardization in this example) on numerical features only
scaler = StandardScaler()
scaled_features_non_nan = scaler.fit_transform(features_non_nan)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    scaled_features_non_nan, target_non_nan, test_size=0.2, random_state=42
)

# Define a simple neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="linear"))

# Compile the model

##mean absolute error 20403 with 100 epochs and 16 batch size
model.compile(optimizer="adam", loss="mean_absolute_error")

# mean absolute error 20507 with 100 epochs and 32 batch size
# mean absolute error 20051 with 150 epochs and 16 batch size

# model.compile(optimizer='rmsprop', loss='mean_absolute_error')

# Train the model and measure time per epoch

early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)

# Train the model with modified parameters

history = model.fit(
    X_train,
    y_train,
    epochs=150,
    batch_size=8,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stopping],
)

# history = model.fit(X_train, y_train, epochs=300, batch_size=8, validation_split=0.2, verbose=1, callbacks=[early_stopping])


# history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)


# Evaluate the model on the test set
y_pred = model.predict(X_test).flatten()

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
medae = median_absolute_error(y_test, y_pred)

# Print the results
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Median Absolute Error (MedAE): {medae}")

# Plot the training history
plt.plot(history.history["loss"], label="train loss")
plt.plot(history.history["val_loss"], label="validation loss")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training History")
plt.show()

# Scatter plot of predicted vs. real OMS
plt.scatter(y_test, y_pred)
plt.xlabel("Real Sales Turnover")
plt.ylabel("Predicted Sales Turnover")
plt.title("Predicted vs. Real Sales Turnover")
plt.show()

# Optional: Residual plot
plt.scatter(y_test, y_pred - y_test)
plt.xlabel("Real Sales Turnover")
plt.ylabel("Residuals (Predicted - Real)")
plt.title("Residual Plot")
plt.show()

# Get rows with NaN values in 'new_oms'
nan_mask = target.isna()
features_nan = features[nan_mask]

# Scale the features
scaled_features_nan = scaler.transform(features_nan)

# Predict missing 'new_oms' values
predicted_nan = model.predict(scaled_features_nan).flatten()

x_resid = X.copy()

# Impute the predicted values back into the original dataframe
X.loc[nan_mask, "new_oms"] = predicted_nan

# Verify the imputation
print(X.head())
print(X[nan_mask].head())  # Check the rows where NaN values were imputed

In [ ]:
X.head(100)

nan_count = X["new_oms"].isna().sum()
print(f"The number of NaN values in the 'new_oms' column is: {nan_count}")

# find outliers - only for test set

In [ ]:
# Predict missing 'new_oms' values for the entire dataset
scaled_features = scaler.transform(features)
predicted_oms = model.predict(scaled_features).flatten()

# Create a new DataFrame with original data and predicted values
# x_resid = X.copy()
x_resid["predicted_oms"] = predicted_oms

# Verify the new DataFrame
x_resid.head()

x_resid["residuals"] = np.abs(x_resid["predicted_oms"] - x_resid["real_oms"])

# x_resid['predicted_oms'] = x_resid['predicted_oms'].round(0).astype(int)
# x_resid['residuals'] = x_resid['residuals'].round(0).astype(int)

x_resid_sorted = x_resid.sort_values(by="residuals", ascending=False)

In [ ]:
x_resid_sorted_subset = x_resid_sorted.head(20)
x_resid_sorted_subset["predicted_oms"] = (
    x_resid_sorted_subset["predicted_oms"].round(0).astype(int)
)
x_resid_sorted_subset["residuals"] = (
    x_resid_sorted_subset["residuals"].round(0).astype(int)
)
x_resid_sorted_subset.head(20)

In [ ]:
# Assuming 'common_column' is the column in both DataFrames to merge on
merged_df_filtered = pd.merge(
    merged_df, x_resid_sorted_subset, on="v_orgnr", how="inner"
)

merged_df_filtered["predicted_oms"] = (
    merged_df_filtered["predicted_oms"].round(0).astype(int)
)
merged_df_filtered["residuals"] = merged_df_filtered["residuals"].round(0).astype(int)
merged_df_filtered = merged_df_filtered.sort_values(by="residuals", ascending=False)
merged_df_filtered.head(20)

In [ ]:
merged_df_filtered.dtypes

# Training History Plot
1. Training Loss and Validation Loss Decrease:

 - Both the training loss and validation loss show a decreasing trend over the epochs, indicating that the model is learning from the data.
 - The training loss starts higher than the validation loss and both decrease gradually, which is a good sign of learning without immediate overfitting.

2. Gap Between Training and Validation Loss:

- The validation loss is consistently lower than the training loss. This could suggest that regularization (e.g., dropout) is effectively preventing overfitting.
- Ideally, the training loss should be slightly lower or close to the validation loss. A significant gap may indicate underfitting or an overly regularized model, but in this case, the gap isn't large enough to cause concern.

# Predcited vs Real Sales Turnover Plot

1. Cluster at Low Values:

- There is a significant clustering of points at lower values of sales turnover, indicating many of your data points are in this range.
- This is common if the majority of your data points have low sales turnover.

2. Deviation at Higher Values:

- The predictions start to deviate more from the true values as the real sales turnover increases. This indicates the model may be struggling to predict higher values accurately. (perhaps not a problem if we continue to collect breakdowns for higher turnover companies)
- This kind of pattern is common in regression tasks with a wide range of target values. The model tends to be more accurate in the range where most training data points are located and less accurate in sparser regions.

# Analysis of the Residual Plot

1. Residuals Close to Zero for Lower Values:

- A significant number of residuals are clustered around zero for lower values of sales turnover. This indicates that the model is performing well for these predictions, with errors being relatively small.

2. Increased Variability at Higher Values:

- As sales turnover increases, the residuals show more variability and are spread out more. This suggests that the model's performance decreases as the sales turnover value increases, resulting in larger prediction errors.

3. Symmetry of Residuals:

- The residuals appear to be fairly symmetrically distributed around zero. This indicates that the model does not have a systematic bias in its predictions (i.e., it does not consistently overpredict or underpredict across the entire range of values).

4. Presence of Outliers:

- There are some points far from the zero line, which can be considered outliers. These indicate instances where the model's predictions are significantly off from the actual values.
- Outliers may indicate that there are some anomalies in the data or that the model struggles with certain patterns in the data.


# Recommendations
1. Data Imbalance:

- If the data is heavily imbalanced (many low turnover values and few high turnover values), consider techniques like resampling, synthetic data generation (SMOTE), or applying different evaluation metrics that are robust to imbalances.

2. Model Complexity:

- Experiment with more complex models or ensemble methods (e.g., Random Forests, Gradient Boosting) to see if they handle higher values better.

3. Feature Engineering:

- Investigate additional feature engineering. Sometimes, additional or transformed features can help the model capture the underlying patterns better.

4. Hyperparameter Tuning:

- Perform hyperparameter tuning using techniques like grid search or random search to optimize the model parameters.

5. Alternative Models:

- Consider using alternative models or architectures that might be better suited for the specific characteristics of your data.

6. Residual Analysis:

- Analyze residuals to understand where the model is performing poorly. Look for patterns or systematic errors in the residuals to guide further improvements.

7. Addressing High Variability at Higher Values:

- Data Transformation: Apply a logarithmic transformation to the sales turnover values to reduce skewness and make the model less sensitive to high values.
- Advanced Models: Try more complex models or ensemble methods like Random Forests, Gradient Boosting, or XGBoost, which might handle the variability better.
- Feature Engineering: Introduce new features or polynomial features that might capture the underlying patterns more effectively.

8. Handling Outliers:

- Outlier Detection and Removal: Identify and remove or treat outliers before training the model. Techniques like IQR (Interquartile Range) or z-score can help in detecting outliers.
- Robust Models: Use models that are robust to outliers, such as robust regression techniques.

9. Model tuning

- Regularization: Apply regularization techniques to avoid overfitting, which can help in better generalization to unseen data.

10. Cross- Validation

- Use cross-validation to ensure that the model's performance is consistent across different subsets of the data.

# Conclusion
- The model is learning and reducing the error over epochs, which is a positive sign. However, its performance in predicting higher sales turnover values indicates that there is room for improvement, possibly through more sophisticated techniques or additional data preprocessing.

- The residual plot suggests that the model performs reasonably well for lower values of sales turnover but struggles with higher values, leading to higher prediction errors. Implementing the recommended improvements could help enhance the model's performance, especially in handling higher sales turnover values and reducing the prediction errors.

<a id='onlygoodoms'></a>
# 1. Only Good Oms DF

#### Uses the profit margin

In [ ]:
onlygoodoms["oms_share"] = onlygoodoms["omsetn_kr"] / onlygoodoms[
    "tot_oms_fordelt"
].round(5)

# Round the values to whole numbers before assigning to the new columns
onlygoodoms["new_oms"] = (
    (onlygoodoms["oms_share"] * onlygoodoms["foretak_omsetning"]).round(0).astype(int)
)

onlygoodoms["margin"] = (
    onlygoodoms["foretak_driftskostnad"] / onlygoodoms["foretak_omsetning"]
)

onlygoodoms["old_drkost"] = onlygoodoms["driftskost_kr"]

onlygoodoms["new_drkost"] = onlygoodoms["margin"] * onlygoodoms["new_oms"]

onlygoodoms["driftskost_kr"] = onlygoodoms["new_drkost"]

onlygoodoms.head()

<a id='gooddf'></a>
# 4. Good DF AO

### rette oms

In [ ]:
# pd.set_option("display.max_rows", None)


good_df["oms_share"] = good_df["omsetn_kr"] / good_df["tot_oms_fordelt"].round(5)

# Round the values to whole numbers before assigning to the new columns
good_df["new_oms"] = (
    (good_df["oms_share"] * good_df["foretak_omsetning"]).round(0).astype(int)
)


# good_df.head()

### rette salgsint og forbruk

data salgsint_forbruk;
	set enhetene_brukes6  
		(
		where=(rad_nr > 0)
		keep=
		orgnr_foretak 
		lopenr
		rad_nr
		tmp_ny_omsetn
		salgsint
		forbruk
		naring
		foretaksnaring
			);
	vhbed=0;

	if substr(naring,1,2) in &w_naring_vh. then
		vhbed=1;

	if naring in &w_nace1_ikke_vh. then
		vhbed=0;

	if substr(naring,1,4) in &w_nace2_ikke_vh. then
		vhbed=0;
run;

w_naring_vh = ('45', '46', '47')
w_nace1_ikke_vh = 45.403
w_nace2_ikke_vh =  ('45.2', '46.1')


In [ ]:
# 1
# Define the values for w_naring_vh, w_nace1_ikke_vh, and w_nace2_ikke_vh
w_naring_vh = ("45", "46", "47")
w_nace1_ikke_vh = "45.403"
w_nace2_ikke_vh = ("45.2", "46.1")

enhetene_brukes = good_df.copy()

# Filter the DataFrame based on conditions and create vhbed variable
enhetene_brukes["vhbed"] = 0

# Check if the first two characters of 'naring' are in w_naring_vh
enhetene_brukes.loc[
    enhetene_brukes["tmp_sn2007_5"].str[:2].isin(w_naring_vh), "vhbed"
] = 1

# Check if 'naring' is in w_nace1_ikke_vh
enhetene_brukes.loc[enhetene_brukes["tmp_sn2007_5"] == w_nace1_ikke_vh, "vhbed"] = 0

# Check if the first four characters of 'naring' are in w_nace2_ikke_vh
enhetene_brukes.loc[
    enhetene_brukes["tmp_sn2007_5"].str[:4].isin(w_nace2_ikke_vh), "vhbed"
] = 0

# enhetene_brukes.head()

In [ ]:
# 2
salgsint_forbruk = enhetene_brukes[
    [
        "orgnr_n_1",
        "lopenr",
        "v_orgnr",
        "forbruk",
        "salgsint",
        "radnr",
        "nacef_5",
        "tmp_sn2007_5",
        "new_oms",
        "vhbed",
    ]
]

# salgsint_forbruk.head()

In [ ]:
# 3

har = salgsint_forbruk[salgsint_forbruk.groupby("orgnr_n_1")["vhbed"].transform("any")]
# Extract the 'orgnr_n_1' column
har = har[["orgnr_n_1"]]

# Remove duplicates
har.drop_duplicates(inplace=True)

# har.head()

In [ ]:
# 4

ikke_har = salgsint_forbruk[
    ~salgsint_forbruk.groupby("orgnr_n_1")["vhbed"].transform("any")
]
ikke_har = ikke_har[["orgnr_n_1"]]
ikke_har.drop_duplicates(inplace=True)

ikke_har["ikkevbed"] = 1

# ikke_har.head()

In [ ]:
# 5


# Merge ikke_har into salgsint_forbruk with a left join on the 'id' column
salgsint_forbruk_update1 = pd.merge(
    salgsint_forbruk, ikke_har, on="orgnr_n_1", how="left"
)

# salgsint_forbruk_update1['ikkevbed'].fillna(0, inplace=True)

# Update 'vhbed' to 1 where 'ikkevbed' is 1
salgsint_forbruk_update1.loc[salgsint_forbruk_update1["ikkevbed"] == 1, "vhbed"] = 1

# salgsint_forbruk_update1.head()

In [ ]:
# 6
# Assuming your original DataFrame is named salgsint_forbruk_update1
# Replace 'new_oms', 'orgnr_foretak', 'lopnr', 'vhbed' with the actual column names in your DataFrame

# Create sum1 DataFrame for vhbed=1
sum1 = (
    salgsint_forbruk_update1[salgsint_forbruk_update1["vhbed"] == 1]
    .groupby(["orgnr_n_1", "lopenr"])["new_oms"]
    .sum()
    .reset_index()
)
sum1.rename(columns={"new_oms": "sumoms_vh"}, inplace=True)

# Create sum2 DataFrame for vhbed=0
sum2 = (
    salgsint_forbruk_update1[salgsint_forbruk_update1["vhbed"] == 0]
    .groupby(["orgnr_n_1", "lopenr"])["new_oms"]
    .sum()
    .reset_index()
)
sum2.rename(columns={"new_oms": "sumoms_andre"}, inplace=True)

In [ ]:
# Perform an outer join of sum1 and sum2 by 'orgnr_foretak' and 'lopnr'
sum3 = pd.merge(sum1, sum2, on=["orgnr_n_1", "lopenr"], how="outer")
# sum3.head()

In [ ]:
# 8

salgsint_forbruk_update2 = pd.merge(
    salgsint_forbruk_update1, sum3, on=["orgnr_n_1", "lopenr"], how="outer"
)
# salgsint_forbruk_update2.head()

In [ ]:
# 9 & 10
# Assuming 'orgnr_n_1', 'lopenr', and 'rad_nr' are the actual column names
# Replace them with the actual names in your DataFrame

# Sort the DataFrame by 'orgnr_n_1', 'lopenr', and 'rad_nr'
salgsint_forbruk_update2.sort_values(by=["orgnr_n_1", "lopenr", "radnr"], inplace=True)

salgsint_forbruk_update2.sort_values(by=["orgnr_n_1", "lopenr", "vhbed"], inplace=True)

# Display the sorted DataFrame
# salgsint_forbruk_update2.head()

In [ ]:
# 11

# Assuming your DataFrame is named salgsint_forbruk
# Replace 'orgnr_foretak', 'lopenr', 'vhbed', 'vhf' with the actual column names

# Sort the DataFrame by 'orgnr_foretak' and 'lopenr'

salgsint_forbruk_update3 = salgsint_forbruk_update2.copy()

salgsint_forbruk_update3.sort_values(by=["orgnr_n_1", "lopenr"], inplace=True)

# Create a new variable 'vhf' based on the values of 'vhbed'
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3.groupby(
    ["orgnr_n_1", "lopenr"]
)["vhbed"].transform("first")

# Retain the value of 'vhf' from the first observation in each group
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3.groupby(
    ["orgnr_n_1", "lopenr"]
)["vhf"].transform("first")

# Apply labels to the variables
salgsint_forbruk_update3["vhbed"] = salgsint_forbruk_update3["vhbed"].astype(str)
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3["vhf"].astype(str)

label_map_vhbed = {"1": "varehandelsbedrift", "0": "annen type bedrift"}
label_map_vhf = {
    "1": "foretaket har kun varehandelsbedrifter eller ingen",
    "0": "har varehandel og annen bedrift (blandingsnæringer)",
}

salgsint_forbruk_update3["vhbed"] = salgsint_forbruk_update3["vhbed"].map(
    label_map_vhbed
)
salgsint_forbruk_update3["vhf"] = salgsint_forbruk_update3["vhf"].map(label_map_vhf)

# salgsint_forbruk_update3.head()

In [ ]:
# 11

# Assuming your DataFrame is named salgsint_forbruk
# Replace 'vhf' with the actual name of your column

# Filter rows where vhf is 'foretaket har kun varehandelsbedrifter eller ingen'
vhf_condition = (
    salgsint_forbruk_update3["vhf"]
    == "foretaket har kun varehandelsbedrifter eller ingen"
)
vhf_df = salgsint_forbruk_update3.loc[vhf_condition]

# Filter rows where vhf is not 'foretaket har kun varehandelsbedrifter eller ingen'
andre_df = salgsint_forbruk_update3.loc[~vhf_condition]

# vhf_df.head()
# andre_df.head()

In [ ]:
# 12

vhf_df["nokkel"] = vhf_df["new_oms"] / vhf_df["sumoms_vh"]

# Convert 'salgsint' column to numeric
vhf_df["salgsint"] = pd.to_numeric(vhf_df["salgsint"], errors="coerce")
vhf_df["forbruk"] = pd.to_numeric(vhf_df["forbruk"], errors="coerce")


vhf_df["bedr_salgsint"] = round(vhf_df["salgsint"] * vhf_df["nokkel"])
vhf_df["bedr_forbruk"] = round(vhf_df["forbruk"] * vhf_df["nokkel"])

# vhf_df.head()

In [ ]:
# 13

andre_df["forbruk"] = pd.to_numeric(andre_df["forbruk"], errors="coerce")
andre_df["salgsint"] = pd.to_numeric(andre_df["salgsint"], errors="coerce")


# Assuming 'andre' is your DataFrame
andre_df["avanse"] = andre_df["forbruk"] / andre_df["salgsint"]

# Filter rows where vhbed is 1
vh_bedriftene = andre_df[andre_df["vhbed"] == "varehandelsbedrift"].copy()

# Calculate 'nokkel', 'bedr_salgsint', and 'bedr_forbruk' for vh-bedriftene
vh_bedriftene["nokkel"] = vh_bedriftene["new_oms"] / vh_bedriftene["sumoms_vh"]
vh_bedriftene["bedr_salgsint"] = round(
    vh_bedriftene["salgsint"] * vh_bedriftene["nokkel"]
)
vh_bedriftene.loc[
    vh_bedriftene["bedr_salgsint"] > vh_bedriftene["new_oms"], "bedr_salgsint"
] = vh_bedriftene["new_oms"]
vh_bedriftene["bedr_forbruk"] = round(
    vh_bedriftene["bedr_salgsint"] * vh_bedriftene["avanse"]
)

# Summarize vh-bedriftene
brukt1 = (
    vh_bedriftene.groupby(["orgnr_n_1", "lopenr"])
    .agg({"bedr_salgsint": "sum", "bedr_forbruk": "sum"})
    .reset_index()
)

# Merge summarized values back to 'andre'
andre = pd.merge(andre_df, brukt1, on=["orgnr_n_1", "lopenr"], how="left")

# Calculate 'resten1' and 'resten2'
andre["resten1"] = andre["salgsint"] - andre["bedr_salgsint"]
andre["resten2"] = andre["forbruk"] - andre["bedr_forbruk"]

# Filter rows where vhbed is not 1
blanding_av_vh_og_andre = andre[andre["vhbed"] != "varehandelsbedrift"].copy()

# Calculate 'nokkel', 'bedr_salgsint', and 'bedr_forbruk' for blending of vh and other industries
blanding_av_vh_og_andre["nokkel"] = (
    blanding_av_vh_og_andre["new_oms"] / blanding_av_vh_og_andre["sumoms_andre"]
)
blanding_av_vh_og_andre["bedr_salgsint"] = round(
    blanding_av_vh_og_andre["resten1"] * blanding_av_vh_og_andre["nokkel"]
)
blanding_av_vh_og_andre["bedr_forbruk"] = round(
    blanding_av_vh_og_andre["resten2"] * blanding_av_vh_og_andre["nokkel"]
)

# Combine the two subsets back into 'andre'
andre = pd.concat([vh_bedriftene, blanding_av_vh_og_andre], ignore_index=True)

andre.sort_values(by=["orgnr_n_1", "lopenr"], inplace=True)

oppdatere_hv = pd.concat([vhf_df, andre], ignore_index=True)

oppdatere_hv = oppdatere_hv[
    ["orgnr_n_1", "lopenr", "radnr", "bedr_forbruk", "bedr_salgsint"]
]

# oppdatere_hv.head()

In [ ]:
enhetene_brukes2 = pd.merge(
    enhetene_brukes, oppdatere_hv, on=["orgnr_n_1", "lopenr", "radnr"]
)

# enhetene_brukes2.head()

In [ ]:
rettes = enhetene_brukes2.copy()

rettes["oms"] = rettes["new_oms"]
rettes["driftsk"] = rettes["gjeldende_driftsk_kr"]

# Convert columns to numeric
rettes["tot_driftskost_fordelt"] = pd.to_numeric(
    rettes["tot_driftskost_fordelt"], errors="coerce"
)
rettes["driftsk"] = pd.to_numeric(rettes["driftsk"], errors="coerce")


rettes["drkost_share"] = rettes["driftsk"] / rettes["tot_driftskost_fordelt"]

rettes["new_drkost"] = rettes["drkost_share"] * rettes["foretak_driftskostnad"]

# rettes.head()

# Kontrol Drkost for forbuk og lønn. 

In [ ]:
rettes2 = rettes.copy()
rettes2["drkost_temp"] = rettes2["new_drkost"]

# Fill NaN in 'drkost_temp' with 0
rettes2["drkost_temp"] = rettes2["drkost_temp"].fillna(0)

rettes2["gjeldende_lonn_kr"] = pd.to_numeric(
    rettes2["gjeldende_lonn_kr"], errors="coerce"
).fillna(0)
rettes2["bedr_forbruk"] = pd.to_numeric(
    rettes2["bedr_forbruk"], errors="coerce"
).fillna(0)


rettes2["lonn_+_forbruk"] = rettes2["gjeldende_lonn_kr"] + rettes2["bedr_forbruk"]

# Perform the if operation
condition = rettes2["drkost_temp"] < rettes2["lonn_+_forbruk"]
rettes2["drkost_temp"] = np.where(
    condition, rettes2["lonn_+_forbruk"], rettes2["drkost_temp"]
)
rettes2["theif"] = np.where(condition, 1, 0)

# rettes2.head()

### Create df with a list of orgnr_foretak that have thieves. 

In [ ]:
# Filter the DataFrame
dkvars = rettes2[rettes2.groupby("orgnr_n_1")["theif"].transform("any")]

# dkvars.shape

In [ ]:
# Calculate 'utskudd'
dkvars["utskudd"] = (
    dkvars["new_drkost"] - dkvars["gjeldende_lonn_kr"] - dkvars["bedr_forbruk"]
)
dkvars["utskudd"] = abs(dkvars["utskudd"])

# Keep selected columns
columns_to_keep = [
    "orgnr_n_1",
    "lopenr",
    "radnr",
    "utskudd",
    "new_drkost",
    "drkost_temp",
    "theif",
    "gjeldende_lonn_kr",
    "bedr_forbruk",
]
dkvars = dkvars[columns_to_keep]

# Display the resulting DataFrame
# dkvars.head()

In [ ]:
# Calculate sum of 'utskudd' grouped by 'orgnr_foretak', 'lopenr', and 'tyv'
sum7b = dkvars.groupby(["orgnr_n_1", "lopenr", "theif"])["utskudd"].sum().reset_index()

# Display the result
# sum7b.head()

In [ ]:
# Transpose the result
sum7b_transposed = sum7b.pivot(
    index=["orgnr_n_1", "lopenr"], columns="theif", values="utskudd"
).reset_index()

# Rename columns as per SAS code
sum7b_transposed.rename(columns={0: "thief0", 1: "thief1"}, inplace=True)

sum7b_transposed = sum7b_transposed[["orgnr_n_1", "lopenr", "thief0", "thief1"]]

# Display the transposed result
# sum7b_transposed.head()

In [ ]:
# merge sums
dkvars_2 = pd.merge(dkvars, sum7b_transposed, on=["orgnr_n_1", "lopenr"], how="inner")
# dkvars_2.head()

In [ ]:
# Apply conditional logic
pd.set_option("display.float_format", "{:.2f}".format)
dkvars_2["andel1"] = np.where(
    dkvars_2["theif"] == 0, dkvars_2["utskudd"] / dkvars_2["thief0"], np.nan
)
dkvars_2["andel2"] = np.where(
    dkvars_2["theif"] == 0, np.round(dkvars_2["andel1"] * dkvars_2["thief1"]), np.nan
)
# dkvars_2['new_drkost'] = np.where(dkvars_2['theif'] == 0, np.sum(dkvars_2['drkost_temp'] - dkvars_2['andel2'], axis=0), dkvars_2['drkost_temp'])
dkvars_2["new_drkost"] = np.where(
    dkvars_2["theif"] == 0,
    dkvars_2["drkost_temp"] - dkvars_2["andel2"],
    dkvars_2["drkost_temp"],
)

# Keep selected columns
columns_to_keep = ["orgnr_n_1", "lopenr", "radnr", "new_drkost"]
dkvars_3 = dkvars_2[columns_to_keep]

# dkvars_2.head(50)

good_final = dkvars_2.copy()

Steps:

1. Define the varehandel næringer that need salgsint and forbruk. 
2. Create a df called salgsint_forbruk that has the forbruk of the foretak, the salgsint of the foretak, orgnr_foretak, rad_nr, naring, foretaksnaring, the new oms and a column called vhbed which indicates 1 if it is a varehandel bedrift and 0 if it is not. 
3. create another df called har that is simply a list of orgnr_foretak where there is at least one bedrift that is in varehandel. Remove duplicates. 
4. create another df called ikke_har that is a list of orgnr_foretak that has 0 bedrifter in varehandel. Create a new variable called ikkevbed = 1. Remove duplicates. 
5. Update salgint_forbruk by merging salgsint_forbruk and ikke with a left join. If ikkevhbed=1 then vhbed=1. Basically what is happenign is that if a foretak has no varehandel, then salgsint will be distributed without bias to varehandel bedrifter. 
6. Create two new dfs. One is called sum1. This sums up all new_oms grouped by orgnr_foretak and lopnr for only if vbed=1. This new variable will be called sumoms_vh. The second df will be called sum2 and will be for only where vhbed=0. This new sum will be a variable called sumoms_andre. 
7. create a new df called sum3 that merges sum1 and sum2 by orgnr_foretak and lopnr. Outerjoin. 
8. update salgsint_forbruk by performing an outerjoin with sum3 by orgnr_foretak and lopnr. Outerjoin. 
9. Sort by orgnr_foretak, lopenr and rad_nr.
10. VERY IMPORTANt. SORT BY ORGNR_FORETAK , LOPENR and VHBED. 
11. data salgsint_forbruk;
	set salgsint_forbruk;
	by orgnr_foretak lopenr;

	if first.orgnr_foretak and first.lopenr then
		vhf=vhbed;
	retain vhf;
	label
		vhbed ='1=varehandelsbedrift, 0 er annen type bedrift'
		vhf   ='1=foretaket har kun varehandelsbedrifter eller ingen , 0=har varehandel og annen bedrift (blandingsnæringer)'
	;
run;

proc sort data=salgsint_forbruk;
	by orgnr_foretak lopenr rad_nr;
run;

*(6) deler opp i rene varehandelsforetak og i andre fioretak;
data vhf  andre;
	set salgsint_forbruk;

	if vhf=1 then
		output vhf;
	else output andre;
run;

Basically what is happening here, is that we are creating two tables. One that has foretak that ONLY have varehandel bedrift OR foretak that have 0 varehandle bedrifter. 

The two tables will be called vhf and andre. 

12. for vhf distribute salgsint and forbruk :

data vhf;
	set vhf;
	nokkel = tmp_ny_omsetn / sumoms_vh;
	bedr_salgsint = round(salgsint * nokkel);
	bedr_forbruk  = round(forbruk * nokkel);
run;

13. for andre. 

data andre;
	set andre;
	avanse=forbruk / salgsint;

	*vh-bedriftene;
	if vhbed=1 then
		do;
			nokkel = tmp_ny_omsetn / sumoms_vh;
			bedr_salgsint = round(salgsint * nokkel);

			if bedr_salgsint > tmp_ny_omsetn then
				bedr_salgsint = tmp_ny_omsetn;
			bedr_forbruk  = round(bedr_salgsint * avanse);
		end;
run;

*(8b) summerere vh-bedriftene;
proc sql stimer;
	create table brukt1 
		as select orgnr_foretak, lopenr, sum(bedr_salgsint) as brukt1, sum(bedr_forbruk) as brukt2
			from andre 
				where vhbed=1
					group by orgnr_foretak, lopenr
						order by orgnr_foretak, lopenr
	;
	create table andre  
		as select a.*, b.*
			from andre  a, brukt1  b
				where a.orgnr_foretak = b.orgnr_foretak and a.lopenr = b.lopenr
					order by a.orgnr_foretak, a.lopenr, a.rad_nr
	;
quit;

*(8c) sluttfordeler for de foretakene som har blanding av vh og andre næringer;
data andre;
	set andre;
	resten1 = sum(salgsint, -brukt1);
	resten2 = sum(forbruk,-brukt2);

	if vhbed ne 1 then
		do;
			nokkel = tmp_ny_omsetn / sumoms_andre;
			bedr_salgsint = round(resten1 * nokkel);
			bedr_forbruk  = round(resten2 * nokkel);
		end;
run;

# What is left to do:

### Correct salgsint and forbruk for good_df. (instructions in cell above) Look at the program in sas as well to know how to do it. 

### Correct drkost for good_df. Have to make sure it passes with salary and forbruk 

### Do the same for the mixed_dfs. (good oms, bad costs etc etc ) Will need to selectivly treat based on what is good and what isnt. Maybe its worth just fixing bad oms. Bad costs can be fixed based on oms

### Implement a the ml models for the bad_df. Maybe make it so you can choose which one to use. 